In [1]:
%%capture
!git clone https://github.com/ronakdm/input-marginalization.git

In [2]:
%%bash
cd input-marginalization
git pull
cd ..

Updating b24eb82..7233752
Fast-forward
 generative_model/transformer-wikitext2.ipynb | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)


From https://github.com/ronakdm/input-marginalization
   b24eb82..7233752  main       -> origin/main


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
save_dir = "/content/gdrive/My Drive/input-marginalization"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
%%capture
!pip install transformers

In [5]:
import os,sys,time,math,textwrap

import numpy as np
import torch
import torch.nn as nn

sys.path.append("input-marginalization/generative_model")
# import dataset, transformer
import transformer
from gen_model_utils import WikiText2Dataset

# root = 'input-marginalization/generative_model/data/wikitext-2'
root = 'input-marginalization/generative_model/data/wikitext-2-raw'

In [18]:
lr = .00035
context = 150
batch_size = 32
log_interval = 50

heads = 8
depth = 8

torch.manual_seed(0)
device = torch.device("cuda")

In [7]:
# train_data = dataset.WikiText2(root, context, dataset.DatasetSplit.train)
# valid_data = dataset.WikiText2(root, context, dataset.DatasetSplit.valid)
# test_data = dataset.WikiText2(root, context, dataset.DatasetSplit.test)

train_data = WikiText2Dataset(file_path=root+"/wiki.train.raw", seq_len=context)
valid_data = WikiText2Dataset(file_path=root+"/wiki.valid.raw", seq_len=context)
test_data = WikiText2Dataset(file_path=root+"/wiki.test.raw", seq_len=context)

Token indices sequence length is longer than the specified maximum sequence length for this model (2303697 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (238658 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (273180 > 512). Running this sequence through the model will result in indexing errors


In [8]:
def evaluate(data):
    model.eval()
    with torch.no_grad():
        loss = 0.
        loader = torch.utils.data.DataLoader(dataset=data,batch_size=batch_size,shuffle=False)
        for i, (x,y) in enumerate(loader):
            x, y = x.permute(1,0).to(device), y.permute(1,0).to(device)
            yhat = model(x).view(-1, train_data.word_count())
            loss += criterion(yhat, y.contiguous().view(-1))

    print()
    model.train()
    return loss / len(loader)

In [19]:
# Real params. TODO: Change back.
# d = 400  # Word embedding dimension.
# k = 40   # Key, query, and value dimension
# m = 900  # Linear layer dimension

d = 200  # Word embedding dimension.
k = 40   # Key, query, and value dimension
m = 500  # Linear layer dimension

# Test params.
# d = 100  # Word embedding dimension.
# k = 20   # Key, query, and value dimension
# m = 200  # Linear layer dimension

model = transformer.Transformer(context, train_data.word_count(), d, k, m, heads, depth, tied_weights=True).to(device)
count = sum([np.prod(parm.shape) for parm in model.parameters() if parm.requires_grad])
print('Initialized graph with {} parameters'.format(count))

Initialized graph with 9824721 parameters


In [20]:
criterion = nn.NLLLoss()
curr_lr = .0001
clip = .25
best_val_loss = None
epochs = 15 # TODO: Change to 10 or 20.
# save = 'model.pt'
save_filename = 'transformer_wikitext2'

train_loader = torch.utils.data.DataLoader(dataset=train_data,batch_size=batch_size,shuffle=True)
print('Initiating training, {} iterations/epoch.'.format(len(train_loader)))

try:
    optimizer = torch.optim.Adam(model.parameters(), lr=curr_lr)
    for epoch in range(epochs):
        t0 = time.time()
        val_loss = evaluate(valid_data)
        print('-' * 100)
        print('| checkpoint | epoch {:3d} | time: {:5.2f}s | validation loss {:5.2f} | '
                'validation perplexity {:8.2f}'.format(epoch, (time.time() - t0),
                                                       val_loss, math.exp(val_loss)))
        print('-' * 100)
        print('epoch\t\tms/batch\tlr\tloss\tperplexity')

        if not best_val_loss or val_loss < best_val_loss:
            # with open(save, 'wb') as f:
            #     torch.save(model, f)
            torch.save(model, f"{save_dir}/{save_filename}.pt") # imarg: Save in Drive folder.
            best_val_loss = val_loss

        model.train()
        total_loss = 0.
        t0 = time.time()
        if epoch == 1: optimizer.param_groups[0]['lr'] = curr_lr = lr # finished warmup
        for i, (x,y) in enumerate(train_loader):
            if i % log_interval == 0 and i > 0:
                cur_loss = total_loss / log_interval
                elapsed = time.time() - t0
                print('{:3d} ({:2.1f}%)\t{:5.2f}\t\t{:1.3}\t{:5.2f}\t{:8.2f}'.format(
                    epoch, 100*i/float(len(train_loader)),
                    elapsed * 1000 / log_interval, curr_lr, cur_loss, math.exp(cur_loss)))
                total_loss = 0
                t0 = time.time()

            x, y = x.permute(1,0).to(device), y.permute(1,0).to(device)
            model.zero_grad()
            yhat = model(x).view(-1, train_data.word_count())
            loss = criterion(yhat, y.contiguous().view(-1))
            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
            optimizer.step()

            total_loss += loss.item()

except KeyboardInterrupt:
    print('Graceful Exit')

Initiating training, 480 iterations/epoch.

----------------------------------------------------------------------------------------------------
| checkpoint | epoch   0 | time:  3.24s | validation loss 10.33 | validation perplexity 30721.20
----------------------------------------------------------------------------------------------------
epoch		ms/batch	lr	loss	perplexity
  0 (10.4%)	191.78		0.0001	 9.56	14128.72
  0 (20.8%)	196.70		0.0001	 8.59	 5386.41
  0 (31.2%)	197.16		0.0001	 7.88	 2633.19
  0 (41.7%)	193.13		0.0001	 7.38	 1600.32
  0 (52.1%)	189.80		0.0001	 7.12	 1230.88
  0 (62.5%)	188.28		0.0001	 6.93	 1025.22
  0 (72.9%)	187.79		0.0001	 6.81	  905.47
  0 (83.3%)	188.10		0.0001	 6.71	  819.74
  0 (93.8%)	189.86		0.0001	 6.63	  757.27

----------------------------------------------------------------------------------------------------
| checkpoint | epoch   1 | time:  3.27s | validation loss  6.58 | validation perplexity   723.83
---------------------------------------------

In [21]:
print('Restoring best checkpointed model...')
# with open(save, 'rb') as f:
#     model = torch.load(f)
model = torch.load(f"{save_dir}/{save_filename}.pt")

test_loss = evaluate(test_data)
print('=' * 89)
print('| end of training | test loss {:5.2f} | test perplexity {:8.2f}'.format(test_loss, math.exp(test_loss)))
print('=' * 89)

Restoring best checkpointed model...

| end of training | test loss  5.33 | test perplexity   205.48


In [22]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

print('\nUncurated samples')
print('-' * 89)

def sample():
    # words = []
    model.eval()

    ids = []
    history = torch.randint(train_data.word_count(), (1, 1), dtype=torch.long).cuda()
    for i in range(context):
        output = model(history)
        word_weights = output[-1].squeeze().exp().cpu()
        word_idx = torch.multinomial(word_weights, 1)[0]
        word_tensor = torch.Tensor([[word_idx]]).long().cuda()
        history = torch.cat([history, word_tensor], 0)

        # words.append(train_data.idx2word[word_idx])
        ids.append(int(word_idx))

    # return '\n'.join(textwrap.wrap(' '.join(words),80))
    return tokenizer.decode(ids)

for i in range(5):
    print('({})'.format(i), sample())


Uncurated samples
-----------------------------------------------------------------------------------------
(0) in testible the devil and the decision and only a object. as the originals researchers tried to decide it and to paratroopers, the jain vision of the trawpitate monofides is the protagonist of more past our governor and its predecessors were attempting to self @ - @ speaking pamsop. the main protagonist is again writing limitation and the intention of the fumots. joyce kishory ghara fighting during euro mesalegs, attacking in which lincolnshire's actions are exceptions, including such a further gling order that he representsive. the tale of this invalid philology and painls with speculation across the street, which had suffered flexible at the ab creek perular metal hotel. the name describes the balance between sentences and vision
(1) any minor athletes at ort county to home home physics. from 2008, a black snakeness crowd held against the majority of the region's sixth sci